A proof of concept showing using a genetic algorithm with our environment.
It is similar to https://github.com/DEAP/deap/blob/a0b78956e28387785e3bb6e2b4b1f1b32c2b3883/examples/ga/onemax_short.py

In [10]:
# Run this cell if you're using colab. Otherwise, skip it.

!git clone https://github.com/platers/meta-transfer-learning.git

import os
os.chdir('meta-transfer-learning')

!pip install -U https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-0.9.0.dev0-cp36-cp36m-manylinux1_x86_64.whl

!pip install -r requirements.txt
!pip install ran


# Make Colab run faster. make sure that the current GPU memory utilization is 0
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Cloning into 'meta-transfer-learning'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 274 (delta 130), reused 216 (delta 83), pack-reused 0
Receiving objects: 100% (274/274), 249.57 KiB | 10.85 MiB/s, done.
Resolving deltas: 100% (130/130), done.
  Using cached https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-0.9.0.dev0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: ray 0.9.0.dev0
    Uninstalling ray-0.9.0.dev0:
      Successfully uninstalled ray-0.9.0.dev0


ERROR: Could not find a version that satisfies the requirement ran (from versions: none)
ERROR: No matching distribution found for ran
Gen RAM Free: 11.5 GB  | Proc size: 1.7 GB
GPU RAM Free: 15891MB | Used: 389MB | Util   2% | Total 16280MB


In [0]:
import array
import random

import numpy as np
from typing import Dict

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import gym

from environments import SimpleEnv
import importlib
from environments import SimpleEnv
importlib.reload(SimpleEnv)

from ray.rllib.agents import ppo
from ray import tune
from ray.rllib.policy.policy import Policy
from ray.rllib.env import BaseEnv
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
from ray.rllib.agents.callbacks import DefaultCallbacks
import ray

import time
start = time.process_time()

from ray import tune

SimpleEnv = SimpleEnv.SimpleEnv

In [0]:
# some settings that we can tweak:
CONFIG_TRAINING={
        "MAX_STEP_COUNT": 1, # number of steps in each round of DEAP evolution 
        "POPULATION_SIZE": 1, # population in each round of DEAP evolution 
        "N_GEN": 1, # number of rounds of DEAP evolution This has the most effect
        "N_RL_TRANING" : 1, # number of rounds of RL training
        "TRAIN_BATCH_SIZE" : 1, #Batch size for RL training
        "LEARNING_RATE" : 0.001 #learning rate for RL training
        }

TUNNING = "RL" # "RL" / "EV" / "OFF". 

if TUNNING == "RL":
  EVAL_METHOD = "RL"  # RL (train reward function and agent) or OPTIMAL (only train reward function)
  # EVAL_METHOD = "OPTIMAL" #-----------need to delete
elif TUNNING == "EV":
  EVAL_METHOD = "OPTIMAL"  # RL (train reward function and agent) or OPTIMAL (only train reward function)
elif TUNNING == "OFF":
  EVAL_METHOD = "RL"  # RL (train reward function and agent) or OPTIMAL (only train reward function)

# List of indivual reward functions used for tunning parameters for evolution
INDIVIDUAL = [[0.9298461960519508, 0.7449587149229808, 0.4628576259710946, -0.282921307700329, -0.7019321146761455, 0.618488821337605]]


In [13]:
ray.shutdown()
ray.init(webui_host='127.0.0.1') #Use this line to fix the error indicated in: https://github.com/ray-project/ray/issues/7084#issuecomment-584505519
n_agents = 3
n_var = 2
training_envs = [
(SimpleEnv, {
    'n_agents': n_agents,
    'n_vars': n_var,
    'true_reward_weights': [1, 0],
    'max_step_count': CONFIG_TRAINING["MAX_STEP_COUNT"],
}),
(SimpleEnv, {
    'n_agents': n_agents,
    'n_vars': n_var,
    'true_reward_weights': [0, 1],
    'max_step_count': CONFIG_TRAINING["MAX_STEP_COUNT"],
})]
test_env = (SimpleEnv, {
    'n_agents': n_agents,
    'n_vars': n_var,
    'true_reward_weights': [1, 1],
    'max_step_count': CONFIG_TRAINING["MAX_STEP_COUNT"],
})



2020-05-09 22:56:30,601	INFO resource_spec.py:212 -- Starting Ray with 7.08 GiB memory available for workers and up to 3.56 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-09 22:56:31,007	INFO services.py:1170 -- View the Ray dashboard at 127.0.0.1:8265


In [0]:
# Directly generate "optimal action" suggested by the reward function created by Evolution
# reward_weights is created from env(config=env_config)
def get_optimal_action(reward_weights, env_config):
    n_vars = env_config["n_vars"]
    max_act = 5    
    
    reward_scale_factor = np.array([2]*n_vars + [1]*(len(reward_weights)-n_vars))
    action = reward_weights * reward_scale_factor
    action = action / np.max(action) * max_act
    
    action = np.reshape(action, [n_agents, n_vars])
    return action

# If we do not want to optimize RL but just want to optimize reward function using Evolution function
# then here we just do the "optimal action" suggested by the generated reward function
def evaluate_individual_env_optimal_act(individual, environment_fn, env_config, config_trianing):
    env_config['reward_weights'] = np.array([individual for i in range(n_agents)])
    env = environment_fn(config=env_config)
    
    ave_true_rewards = 0
    obs = env.reset()
    ave_reward = 0
    for _ in range(env_config["max_step_count"]):
        actions = {i: get_optimal_action(env.reward_weights[i], env_config) for i in range(n_agents)}
        obs, reward, _, _ = env.step(actions)
        reward = np.array([reward[i] for i in range(len(reward))])
        ave_reward += reward
        ave_true_rewards += env.last_true_reward
        
    ave_true_rewards /= env_config["max_step_count"]
    ave_reward /= env_config["max_step_count"]
    
    return np.mean(ave_true_rewards)

In [0]:
# train RL agent with given setup
def evaluate_individual_env_rl(individual, environment, env_config, config_trianing):
    """Runs the environment. All agents have the same policy.
  It returns the total true reward as the fitness.
  """
    # need to import many libaraies again, because otherwise tunning funciton will not find the libaraies. 
    import array
    import random

    import numpy as np
    from typing import Dict

    from deap import algorithms
    from deap import base
    from deap import creator
    from deap import tools
    import gym

    from environments import SimpleEnv
    import importlib
    from environments import SimpleEnv
    importlib.reload(SimpleEnv)

    from ray.rllib.agents import ppo
    from ray import tune
    from ray.rllib.policy.policy import Policy
    from ray.rllib.env import BaseEnv
    from ray.rllib.env.multi_agent_env import MultiAgentEnv
    from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
    from ray.rllib.agents.callbacks import DefaultCallbacks
    import ray

    import time
    start = time.process_time()

    from ray import tune

    SimpleEnv = SimpleEnv.SimpleEnv
    
    import time
    start = time.process_time()
    # your code here    
    print("Time Spent 1--- = ", (time.process_time() - start)/60, " minutes")  
    
    #Select random individuals from pop and create the reward weights
    pop = np.array([individual for i in range(n_agents)])
    reward_weights = pop
    env_config['reward_weights'] = reward_weights
    #env is only to get action space and observation space
    env = environment(config=env_config)
    class MyCallbacks(DefaultCallbacks):
        #Callback functions to keep track of true reward while training
        def on_episode_start(self, worker: RolloutWorker, base_env: BaseEnv,
                         policies: Dict[str, Policy],
                         episode: MultiAgentEpisode, **kwargs):
            episode.user_data["true_rewards"] = np.zeros(n_agents)

        def on_episode_step(self, worker: RolloutWorker, base_env: BaseEnv,
                        episode: MultiAgentEpisode, **kwargs):
            env = base_env
            true_reward = env.env_states[0].env.last_true_reward
            episode.user_data["true_rewards"] += true_reward

        def on_episode_end(self, worker: RolloutWorker, base_env: BaseEnv,
                       policies: Dict[str, Policy], episode: MultiAgentEpisode,
                       **kwargs):
            true_reward = episode.user_data["true_rewards"]
            for i, r in enumerate(true_reward):
                episode.custom_metrics["true_reward_agent_" + str(i)] = r
    
    print("Time Spent 2--- = ", (time.process_time() - start)/60, " minutes")  
    
    
    # settings for the RL agent trainer     
    config={
        "train_batch_size": config_trianing["TRAIN_BATCH_SIZE"],
        "lr": config_trianing["LEARNING_RATE"],
        "sgd_minibatch_size": config_trianing["TRAIN_BATCH_SIZE"],
        "multiagent": {
            "policies": {
            },
            "policy_mapping_fn":  #all agents share a policy
                lambda agent_id:
                    'agent'
        },
        "model": {"fcnet_hiddens": []},
        'env_config': env_config,
        "callbacks": MyCallbacks,
    }
    config['multiagent']['policies']['agent'] = (None, env.observation_space, env.action_space, {})
    metrics = None


    print("Time Spent 3--- = ", (time.process_time() - start)/60, " minutes")  
    
    counter = 0 # ------ need to take this out later. use counter to break.
    while True:
        counter += 1 # ------ need to take this out later. use counter to break.
        trainer = ppo.PPOTrainer(env=environment, config=config)
        true_reward_mean = 0
        print("Time Spent 4--- = ", (time.process_time() - start)/60, " minutes, counter = ", counter)  
        
        for i in range(config_trianing["N_RL_TRANING"]):
            true_reward_mean = 0
            #Train the RL agent
            print("Time Spent 5--(right before trainig)-- = ", (time.process_time() - start)/60, " minutes, counter = ", counter)
            # import pdb; pdb.set_trace()
            print("config_trianing = ", config_trianing, "individual=", individual)

            metrics = trainer.train()  # distributed training step
            print("episode_reward_mean", metrics["episode_reward_mean"])
            print("Time Spent 6--- = ", (time.process_time() - start)/60, " minutes, counter = ", counter)  
            # ------ need to take this out later. use counter to break.
            if counter >= 1:
                break

            if metrics["episode_reward_mean"] < 0 and i > 2:
                break
        #Train agent until it does well
        if metrics["episode_reward_mean"] > 0:
            break
        # ------ need to take this out later. use counter to break.
        if counter >= 1:
            break
    
    print("Time Spent 7--- = ", (time.process_time() - start)/60, " minutes")  
    
    print("episode_reward_mean", metrics["episode_reward_mean"])
    for i in range(n_agents):
        true_reward_mean += metrics['custom_metrics']['true_reward_agent_' + str(i) + '_mean']
    true_reward_mean /= n_agents
    print('Evaluated', individual, 'Fitness', true_reward_mean)

    print("Time Spent 8--- = ", (time.process_time() - start)/60, " minutes")  
    
    return true_reward_mean

In [0]:
if EVAL_METHOD == "RL":
    evaluate_individual_env = evaluate_individual_env_rl
else:
    evaluate_individual_env = evaluate_individual_env_optimal_act
        
def evaluate_individual(individual, config_trianing = CONFIG_TRAINING):
    """Runs all environments. 
  returns the average true reward over all environments as the fitness.
  """
    
    all_fitness = []
    for env, config in training_envs:
        all_fitness.append(evaluate_individual_env(individual, env, config, config_trianing))
    fitness_comb = np.min(all_fitness)
    return (fitness_comb, )

In [0]:
#function that tunes the reward function
def train_reward_function(config_trianing):

  creator.create('FitnessMax', base.Fitness, weights=(1.0, ))
  creator.create('Individual', array.array, typecode='d',
                fitness=creator.FitnessMax)

  toolbox = base.Toolbox()

  toolbox.register('attr', random.randint, 0, 1)
  toolbox.register('individual', tools.initRepeat, creator.Individual,
                  toolbox.attr, n_agents * n_var)
  toolbox.register('population', tools.initRepeat, list,
                  toolbox.individual)
    # some setup for Deap
  toolbox.register('evaluate', evaluate_individual)
  toolbox.register('mate', tools.cxTwoPoint)
  toolbox.register('mutate', tools.mutFlipBit, indpb=0.05) # add more noise
  toolbox.register('select', tools.selTournament, tournsize=3)

  # pop is a list individual reward function's weights 
  pop = toolbox.population(n=config_trianing["POPULATION_SIZE"])
  # the 10 best individual reward function (could even befround the first round)
  hof = tools.HallOfFame(10)

  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register('avg', np.mean)
  stats.register('std', np.std)
  stats.register('min', np.min)
  stats.register('max', np.max)



  
  pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=config_trianing["N_GEN"], 
                                    stats=stats, halloffame=hof, verbose=True)
  best_individual = hof[0]
  print(best_individual)
  test_reward = evaluate_individual_env(best_individual, test_env[0], test_env[1], config_trianing)
  print("---test_reward:", test_reward)
  if TUNNING == "EV":
    tune.track.log(mean_accuracy=test_reward) 

def tune_rl_training(config_trianing):
  print("---rl tunning starts")
  all_score = []
  for individual in INDIVIDUAL:
    all_score.append(evaluate_individual(config_trianing)[0])
  tune.track.log(mean_accuracy=sum(all_score)/len(all_score))
  

if TUNNING == "EV":
  analysis = tune.run(
      train_reward_function, config={
          "N_GEN": tune.grid_search(list(range(1, 20, 5))), 
          "POPULATION_SIZE": tune.grid_search(list(range(1, 20, 5)))
          # "N_GEN": tune.grid_search([1]), 
          # "POPULATION_SIZE": tune.grid_search([1])
          })
  #print best config
  print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))
elif TUNNING == "RL":
  analysis = tune.run(
      tune_rl_training, config={
        "N_RL_TRANING": tune.grid_search([1]),
        "TRAIN_BATCH_SIZE": tune.grid_search([1]),
        "LEARNING_RATE": tune.grid_search([0.001])
      })
  print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))
elif TUNNING == "OFF":
  train_reward_function(CONFIG_TRAINING)



# print ('pop', pop)

2020-05-09 22:56:33,206	WARNING tune.py:316 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,LEARNING_RATE,N_RL_TRANING,TRAIN_BATCH_SIZE
tune_rl_training_53f25_00000,RUNNING,,0.001,1,1


(pid=4241) ---rl tunning starts
(pid=4241) Time Spent 1--- =  1.746666666117373e-08  minutes
(pid=4241) Time Spent 2--- =  8.5957449999996e-05  minutes
(pid=4241) Time Spent 3--- =  8.676783333333068e-05  minutes
(pid=4241) 2020-05-09 22:56:37,114	INFO trainable.py:217 -- Getting current IP.
(pid=4241) 2020-05-09 22:56:37,129	INFO trainer.py:425 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=4241) 2020-05-09 22:56:37,158	INFO trainer.py:584 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=4242) /usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=4242)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=4241) Time Spent 4--- =  0.08420111538333332  minutes, counter =  1
(pid=4241) Time Spent 5--(right before trainig)-- =  0.08420190224999999  minutes, counter =  1
(pid=4241) conf

2020-05-09 22:56:51,310	WARNING worker.py:1094 -- The actor or task with ID ffffffffffffffff19e86ed90100 is pending and cannot currently be scheduled. It requires {CPU: 1.000000} for execution and {CPU: 1.000000} for placement, but this node only has remaining {node:172.28.0.2: 1.000000}, {memory: 7.080078 GiB}, {GPU: 1.000000}, {object_store_memory: 2.441406 GiB}. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale.
2020-05-09 22:56:51,922	INFO (unknown file):0 -- gc.collect() freed 60520 refs in 0.5107968550000805 seconds


(pid=4242) 2020-05-09 22:56:51,649	INFO (unknown file):0 -- gc.collect() freed 16 refs in 0.23870680999993965 seconds


In [0]:
print(evaluate_individual([0.9298461960519508, 0.7449587149229808, 0.4628576259710946, -0.282921307700329, -0.7019321146761455, 0.618488821337605]))

In [0]:
print(evaluate_individual([1, 0, -1, 0, -1, 0])) #Worst reward, selfish agent

In [0]:
print("Time Spent = ", (time.process_time() - start)/60, " minutes")